# **Capstone Project - Simple Chatbot untuk FAQ KitaFit**

Notebook ini membangun chatbot sederhana untuk menjawab FAQ KitaFit menggunakan TensorFlow dan TF-IDF. Tujuan utama adalah untuk membuat model yang dapat memahami pertanyaan pengguna dan memberikan jawaban yang relevan dari daftar FAQ yang telah ditentukan.

### **Import Library dan Inisialisasi Stemmer Sastrawi**

Sel ini bertanggung jawab untuk mengimpor semua library yang kita butuhkan untuk proyek ini. Selain itu, kita juga akan menginisialisasi *stemmer* dari library **Sastrawi**. Sastrawi adalah library stemming yang dirancang khusus untuk Bahasa Indonesia, sehingga memberikan hasil yang jauh lebih baik dibandingkan `PorterStemmer`.

**Catatan:** Pastikan Anda sudah menginstall Sastrawi. Jika belum, jalankan perintah berikut di terminal atau command prompt Anda (dengan virtual environment aktif):
```bash
pip install Sastrawi

In [1]:
import pandas as pd
import numpy as np
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Import NLTK untuk tokenisasi dan stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Import Sastrawi untuk stemming Bahasa Indonesia
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Download resource NLTK yang diperlukan (hanya dijalankan sekali)
try:
    stopwords.words('indonesian')
except LookupError:
    nltk.download('stopwords')
try:
    word_tokenize("test")
except LookupError:
    nltk.download('punkt')

# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Inisialisasi daftar stop words Bahasa Indonesia
indonesian_stop_words = set(stopwords.words('indonesian'))

print("Libraries imported and Sastrawi stemmer is ready.")

Libraries imported and Sastrawi stemmer is ready.


### **Pembuatan Dataset (Data Generation)**

Tujuan dari sel ini adalah untuk membuat dataset awal kita. Kode ini akan menghasilkan sebuah file CSV bernama `faq_dataset_100.csv` yang berisi 100 contoh pertanyaan, masing-masing dengan label "intent" (maksud) yang sesuai.

Jika file `faq_dataset_100.csv` sudah ada di direktori Anda, Anda tidak perlu menjalankan sel ini lagi.

## **Data Generation (Dataset Creation)**

In [2]:
# Blok kode ini membuat dataset.
faq_data = {
    "intent": [],
    "text": []
}

# 10 intents utama
extended_faqs = [
    # kitafit
    {"intent": "kitafit", "text": "Apa itu KitaFit?"},
    {"intent": "kitafit", "text": "KitaFit itu aplikasi apa?"},
    {"intent": "kitafit", "text": "Fungsi utama KitaFit?"},
    {"intent": "kitafit", "text": "Apa manfaat menggunakan KitaFit?"},
    {"intent": "kitafit", "text": "Untuk apa KitaFit dibuat?"},
    {"intent": "kitafit", "text": "Apakah KitaFit aplikasi kesehatan?"},
    {"intent": "kitafit", "text": "Apakah KitaFit hanya untuk jantung?"},
    {"intent": "kitafit", "text": "Apakah KitaFit tersedia di iOS?"},
    {"intent": "kitafit", "text": "Apakah KitaFit tersedia di Android?"},
    {"intent": "kitafit", "text": "Bagaimana KitaFit membantu kesehatan jantung?"},

    # daftar
    {"intent": "daftar", "text": "Bagaimana cara mendaftar di KitaFit?"},
    {"intent": "daftar", "text": "Langkah-langkah pendaftaran KitaFit?"},
    {"intent": "daftar", "text": "Daftar KitaFit itu susah tidak?"},
    {"intent": "daftar", "text": "Apa yang dibutuhkan untuk daftar KitaFit?"},
    {"intent": "daftar", "text": "Bisakah daftar pakai email?"},
    {"intent": "daftar", "text": "Bisakah daftar pakai nomor telepon?"},
    {"intent": "daftar", "text": "Apakah harus verifikasi email setelah daftar?"},
    {"intent": "daftar", "text": "Saya lupa password, bagaimana cara reset?"},
    {"intent": "daftar", "text": "Bagaimana jika saya tidak menerima email verifikasi?"},
    {"intent": "daftar", "text": "Apakah ada batas usia untuk mendaftar?"},

    # fitur
    {"intent": "fitur", "text": "Apa saja fitur yang ada di KitaFit?"},
    {"intent": "fitur", "text": "Fitur unggulan KitaFit apa saja?"},
    {"intent": "fitur", "text": "Jelaskan fitur deteksi risiko di KitaFit." },
    {"intent": "fitur", "text": "Bagaimana fitur konsultasi bekerja?"},
    {"intent": "fitur", "text": "Apa itu rekomendasi gaya hidup di KitaFit?"},
    {"intent": "fitur", "text": "Apakah KitaFit bisa melacak aktivitas olahraga saya?"},
    {"intent": "fitur", "text": "Apakah ada fitur pengingat obat?"},
    {"intent": "fitur", "text": "Apakah KitaFit ada fitur monitoring tidur?"},
    {"intent": "fitur", "text": "Bisakah saya mencatat asupan makanan di KitaFit?"},
    {"intent": "fitur", "text": "Apakah ada fitur komunitas di KitaFit?"},

    # biaya
    {"intent": "biaya", "text": "Berapa biaya menggunakan KitaFit?"},
    {"intent": "biaya", "text": "Apakah KitaFit gratis?"},
    {"intent": "biaya", "text": "Harga langganan KitaFit premium berapa?"},
    {"intent": "biaya", "text": "Apakah ada trial untuk KitaFit premium?"},
    {"intent": "biaya", "text": "Metode pembayaran apa saja yang diterima?"},
    {"intent": "biaya", "text": "Bisakah saya batalkan langganan premium?"},
    {"intent": "biaya", "text": "Apa perbedaan fitur gratis dan premium?"},
    {"intent": "biaya", "text": "Apakah ada diskon untuk langganan tahunan?"},
    {"intent": "biaya", "text": "Bagaimana cara upgrade ke premium?"},
    {"intent": "biaya", "text": "Apakah biaya konsultasi termasuk dalam paket premium?"},

    # riwayat
    {"intent": "riwayat", "text": "Bagaimana cara melihat riwayat kesehatan saya?"},
    {"intent": "riwayat", "text": "Di mana letak menu riwayat di KitaFit?"},
    {"intent": "riwayat", "text": "Bisakah saya mengunduh riwayat kesehatan?"},
    {"intent": "riwayat", "text": "Berapa lama data riwayat saya disimpan?"},
    {"intent": "riwayat", "text": "Bisakah saya menghapus data riwayat tertentu?"},
    {"intent": "riwayat", "text": "Apakah riwayat saya bisa dilihat orang lain?"},
    {"intent": "riwayat", "text": "Bagaimana cara melacak perubahan kondisi saya?"},
    {"intent": "riwayat", "text": "Apakah riwayat saya bisa dibagi dengan dokter?"},
    {"intent": "riwayat", "text": "Apakah riwayat aktivitas olahraga saya tercatat?"},
    {"intent": "riwayat", "text": "Bagaimana cara menambahkan data riwayat manual?"},

    # hasil
    {"intent": "hasil", "text": "Bagaimana hasil deteksi risiko disajikan?"},
    {"intent": "hasil", "text": "Apa arti skor risiko di KitaFit?"},
    {"intent": "hasil", "text": "Seberapa akurat hasil deteksi KitaFit?"},
    {"intent": "hasil", "text": "Apakah saya perlu ke dokter setelah dapat hasil deteksi?"},
    {"intent": "hasil", "text": "Bagaimana cara meningkatkan skor kesehatan saya?"},
    {"intent": "hasil", "text": "Bisakah saya mencetak hasil deteksi?"},
    {"intent": "hasil", "text": "Apakah hasil deteksi memperhitungkan riwayat keluarga?"},
    {"intent": "hasil", "text": "Saya tidak mengerti hasil deteksi saya, bagaimana?"},
    {"intent": "hasil", "text": "Berapa lama waktu yang dibutuhkan untuk mendapatkan hasil deteksi?"},
    {"intent": "hasil", "text": "Apakah hasil deteksi saya bersifat rahasia?"},

    # reset_password
    {"intent": "reset_password", "text": "Bagaimana cara reset password KitaFit?"},
    {"intent": "reset_password", "text": "Saya lupa kata sandi, tolong bantu."},
    {"intent": "reset_password", "text": "Link untuk reset password di mana?"},
    {"intent": "reset_password", "text": "Email reset password tidak sampai."},
    {"intent": "reset_password", "text": "Berapa lama link reset password berlaku?"},
    {"intent": "reset_password", "text": "Bisakah saya mengganti password tanpa reset?"},
    {"intent": "reset_password", "text": "Saya tidak bisa login, padahal password benar."},
    {"intent": "reset_password", "text": "Apakah ada batasan berapa kali bisa reset password?"},
    {"intent": "reset_password", "text": "Bagaimana cara membuat password yang aman?"},
    {"intent": "reset_password", "text": "Saya tidak punya akses ke email/nomor telepon terdaftar."},

    # konsultasi
    {"intent": "konsultasi", "text": "Bagaimana cara konsultasi dengan dokter di KitaFit?"},
    {"intent": "konsultasi", "text": "Apakah saya bisa chat dengan ahli gizi?"},
    {"intent": "konsultasi", "text": "Jenis konsultasi apa saja yang tersedia?"},
    {"intent": "konsultasi", "text": "Berapa lama durasi konsultasi?"},
    {"intent": "konsultasi", "text": "Apakah konsultasi KitaFit 24 jam?"},
    {"intent": "konsultasi", "text": "Bagaimana cara menjadwalkan konsultasi?"},
    {"intent": "konsultasi", "text": "Bisakah saya memilih dokter spesialis jantung?"},
    {"intent": "konsultasi", "text": "Apakah data konsultasi saya aman?"},
    {"intent": "konsultasi", "text": "Saya ingin konsultasi tentang diet jantung, bisakah?"},
    {"intent": "konsultasi", "text": "Bagaimana jika koneksi terputus saat konsultasi video?"},

    # privasi
    {"intent": "privasi", "text": "Bagaimana KitaFit menjaga privasi data saya?"},
    {"intent": "privasi", "text": "Apakah data saya aman di KitaFit?"},
    {"intent": "privasi", "text": "KitaFit membagikan data saya ke pihak ketiga tidak?"},
    {"intent": "privasi", "text": "Apakah KitaFit patuh pada regulasi privasi data?"},
    {"intent": "privasi", "text": "Bagaimana cara menghapus akun dan data saya?"},
    {"intent": "privasi", "text": "Apakah data kesehatan saya terenkripsi?"},
    {"intent": "privasi", "text": "Siapa yang memiliki akses ke data saya?"},
    {"intent": "privasi", "text": "Apa kebijakan privasi KitaFit?"},
    {"intent": "privasi", "text": "Bagaimana cara mengelola izin data saya?"},
    {"intent": "privasi", "text": "Apakah KitaFit menggunakan data saya untuk iklan?"},

    # artikel
    {"intent": "artikel", "text": "Di mana saya bisa menemukan artikel kesehatan?"},
    {"intent": "artikel", "text": "Apa itu LifeNews di KitaFit?"},
    {"intent": "artikel", "text": "Topik artikel apa saja yang tersedia?"},
    {"intent": "artikel", "text": "Apakah artikel KitaFit ditulis oleh ahli?"},
    {"intent": "artikel", "text": "Bisakah saya menyimpan artikel untuk dibaca nanti?"},
    {"intent": "artikel", "text": "Bagaimana cara mencari artikel tertentu?"},
    {"intent": "artikel", "text": "Apakah ada video edukasi di KitaFit?"},
    {"intent": "artikel", "text": "Apakah LifeNews diperbarui secara berkala?"},
    {"intent": "artikel", "text": "Saya ingin menyumbangkan artikel, bisakah?"},
    {"intent": "artikel", "text": "Apakah ada kategori khusus untuk artikel jantung?"}
]

for faq in extended_faqs:
    faq_data["intent"].append(faq["intent"])
    faq_data["text"].append(faq["text"])

df_extended = pd.DataFrame(faq_data)
file_name = "faq_dataset_100.csv"
df_extended.to_csv(file_name, index=False)

print(f"Dataset dengan {len(df_extended)} data telah dibuat: {file_name}.")
print(f"Contoh data:\n{df_extended.head()}")
print(f"\nDistribusi Intent:\n{df_extended['intent'].value_counts()}")

Dataset dengan 100 data telah dibuat: faq_dataset_100.csv.
Contoh data:
    intent                              text
0  kitafit                  Apa itu KitaFit?
1  kitafit         KitaFit itu aplikasi apa?
2  kitafit             Fungsi utama KitaFit?
3  kitafit  Apa manfaat menggunakan KitaFit?
4  kitafit         Untuk apa KitaFit dibuat?

Distribusi Intent:
intent
kitafit           10
daftar            10
fitur             10
biaya             10
riwayat           10
hasil             10
reset_password    10
konsultasi        10
privasi           10
artikel           10
Name: count, dtype: int64


### **Memuat dan Melakukan Preprocessing Data**

Ini adalah salah satu langkah paling penting. Di sini kita akan:
1.  **Memuat** file `faq_dataset_100.csv` yang sudah kita buat.
2.  **Membersihkan Teks**: Membuat fungsi `preprocess_text` untuk membersihkan setiap pertanyaan. Proses ini mencakup:
    -   Mengubah teks menjadi huruf kecil.
    -   Menghapus tanda baca dan karakter spesial.
    -   *Tokenisasi*: Memecah kalimat menjadi token (kata).
    -   Menghapus *stop words* (kata-kata umum yang tidak memiliki banyak makna, seperti 'di', 'dan', 'yang').
    -   *Stemming*: Mengubah setiap kata ke bentuk dasarnya menggunakan **Sastrawi**. Contoh: "mendaftar" -> "daftar".
3.  **Encoding Label**: Mengubah label `intent` yang berupa teks (misal: "fitur") menjadi angka (misal: 3) menggunakan `LabelEncoder` agar bisa diproses oleh model.
4.  **Ekstraksi Fitur**: Mengubah teks yang sudah bersih menjadi vektor numerik menggunakan `TfidfVectorizer`. Model tidak bisa membaca teks, jadi kita harus mengubahnya menjadi angka.
5.  **Membagi Data**: Memisahkan data menjadi data latih (80%) dan data uji (20%) untuk melatih dan mengevaluasi model.

In [3]:
# Memuat dataset FAQ
df = pd.read_csv("faq_dataset_100.csv")

# Fungsi untuk membersihkan teks, menghapus stop words, dan melakukan stemming
def preprocess_text(text):
    # Mengubah ke huruf kecil
    text = text.lower()
    # Menghapus karakter selain huruf dan angka
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Tokenisasi
    words = word_tokenize(text)
    # Menghapus stop words dan melakukan stemming dengan Sastrawi
    processed_words = [stemmer.stem(word) for word in words if word not in indonesian_stop_words and word.isalnum()]
    return ' '.join(processed_words)

# Terapkan preprocessing pada kolom 'text'
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("Contoh teks setelah preprocessing dengan Sastrawi:")
for i in range(3):
    print(f"Original: {df['text'].iloc[i]}")
    print(f"Cleaned : {df['cleaned_text'].iloc[i]}\n")

# Meng-encode label intent menjadi angka
le = LabelEncoder()
df['intent_encoded'] = le.fit_transform(df["intent"])
y = df['intent_encoded']
num_classes = len(le.classes_)
print(f"Jumlah kelas unik (intents): {num_classes}")
print(f"Mapping intent ke encoding:\n{dict(zip(le.classes_, le.transform(le.classes_)))}\n")

# Mengubah teks menjadi fitur numerik menggunakan TF-IDF
tfidf = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
X = tfidf.fit_transform(df["cleaned_text"]).toarray()

# Membagi data menjadi set pelatihan (training) dan pengujian (testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

Contoh teks setelah preprocessing dengan Sastrawi:
Original: Apa itu KitaFit?
Cleaned : kitafit

Original: KitaFit itu aplikasi apa?
Cleaned : kitafit aplikasi

Original: Fungsi utama KitaFit?
Cleaned : fungsi utama kitafit

Jumlah kelas unik (intents): 10
Mapping intent ke encoding:
{'artikel': 0, 'biaya': 1, 'daftar': 2, 'fitur': 3, 'hasil': 4, 'kitafit': 5, 'konsultasi': 6, 'privasi': 7, 'reset_password': 8, 'riwayat': 9}

Shape of X_train: (80, 315)
Shape of X_test: (20, 315)


### **Membangun Arsitektur Model Neural Network**

Pada sel ini, kita mendefinisikan arsitektur dari model *neural network* kita menggunakan Keras. Model ini cukup sederhana dan cocok untuk dataset kita yang kecil.

Arsitekturnya terdiri dari:
-   **Input Layer**: Menerima data dengan dimensi sesuai jumlah fitur dari TF-IDF.
-   **Hidden Layers**: Dua lapisan tersembunyi (`Dense`) dengan fungsi aktivasi `ReLU`. Lapisan ini bertugas untuk mempelajari pola dari data.
-   **Dropout Layers**: Lapisan `Dropout` ditambahkan setelah setiap lapisan tersembunyi untuk mengurangi *overfitting*. Dropout secara acak menonaktifkan beberapa neuron selama pelatihan, memaksa model untuk tidak terlalu bergantung pada neuron tertentu.
-   **Output Layer**: Satu lapisan `Dense` dengan jumlah neuron sebanyak jumlah kelas (intent) dan fungsi aktivasi `softmax`. Softmax akan menghasilkan probabilitas untuk setiap kelas, dan kelas dengan probabilitas tertinggi akan menjadi prediksi kita.

In [4]:
# Parameter model
input_dim = X_train.shape[1]

# Membangun arsitektur model
model = Sequential([
    Input(shape=(input_dim,)),
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Mengkompilasi model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        40,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,354 (192.79 KB)

 Trainable params: 49,354 (192.79 KB)

 Non-trainable params: 0 (0.00 B)

### **Melatih Model**

Sekarang saatnya melatih model kita dengan data training yang sudah disiapkan.

-   `model.fit()` adalah fungsi untuk memulai proses pelatihan.
-   **EarlyStopping** digunakan sebagai *callback*. Ini adalah teknik yang sangat berguna untuk menghentikan pelatihan secara otomatis jika performa model pada data validasi tidak meningkat setelah sejumlah `patience` (misal: 15 epoch). Ini mencegah *overfitting* dan menghemat waktu. `restore_best_weights=True` memastikan kita menggunakan bobot model dari epoch terbaik.

In [5]:
# Callback EarlyStopping untuk mencegah overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

# Melatih model
epochs = 100
batch_size = 8

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    shuffle=True,
    verbose=1
)

print("\nModel training complete.")

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.1445 - loss: 2.3037 - val_accuracy: 0.1500 - val_loss: 2.2976
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1451 - loss: 2.2762 - val_accuracy: 0.1500 - val_loss: 2.2880
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2431 - loss: 2.2537 - val_accuracy: 0.1500 - val_loss: 2.2792
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2602 - loss: 2.2401 - val_accuracy: 0.1500 - val_loss: 2.2658
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3803 - loss: 2.1682 - val_accuracy: 0.2000 - val_loss: 2.2490
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4234 - loss: 2.1581 - val_accuracy: 0.2500 - val_loss: 2.2307
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4878 - loss: 2.0885 - val_accuracy: 0.2500 - val_loss: 2.2103
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5232 - loss: 2.0605 - val_accuracy: 0.400

### **Mengevaluasi Performa Model**

Setelah model dilatih, kita perlu mengukur seberapa baik performanya pada data yang belum pernah ia lihat sebelumnya (data tes).

-   `model.evaluate()` akan memberikan skor `loss` dan `accuracy` akhir dari model pada data tes.
-   `classification_report` dari Scikit-learn memberikan metrik yang lebih rinci untuk setiap kelas, seperti `precision`, `recall`, dan `f1-score`, yang membantu kita memahami di intent mana model berkinerja baik atau buruk.

In [6]:
# Evaluasi model pada set pengujian
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Mendapatkan prediksi probabilitas dan kelas
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Menampilkan Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))


Test Loss: 1.0345
Test Accuracy: 0.7500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

Classification Report:
                precision    recall  f1-score   support

       artikel       1.00      0.50      0.67         2
         biaya       0.67      1.00      0.80         2
        daftar       1.00      1.00      1.00         2
         fitur       1.00      0.50      0.67         2
         hasil       1.00      1.00      1.00         2
       kitafit       0.40      1.00      0.57         2
    konsultasi       1.00      0.50      0.67         2
       privasi       1.00      1.00      1.00         2
reset_password       1.00      0.50      0.67         2
       riwayat       0.50      0.50      0.50         2

      accuracy                           0.75        20
     macro avg       0.86      0.75      0.75        20
  weighted avg       0.86      0.75      0.75        20



### **Implementasi Interaktif Chatbot**

Ini adalah sel terakhir di mana kita menggabungkan semua yang telah kita bangun menjadi sebuah chatbot interaktif.

1.  **`intent_responses`**: Sebuah dictionary Python yang memetakan setiap `intent` ke daftar kemungkinan jawaban. Ini memungkinkan chatbot memberikan jawaban yang bervariasi.
2.  **`get_chatbot_response`**: Fungsi utama yang menerima input dari pengguna, melakukan semua langkah preprocessing, mendapatkan prediksi dari model, dan mengembalikan jawaban yang sesuai.
3.  **Loop `while True`**: Bagian ini membuat chatbot terus berjalan, menerima input dari pengguna, dan memberikan respons sampai pengguna mengetik 'keluar'.

In [8]:
# Kamus respons untuk setiap intent
intent_responses = {
    "kitafit": [
        "KitaFit adalah platform kesehatan berbasis Machine Learning yang dirancang untuk membantu Anda mendeteksi risiko penyakit jantung sejak dini.",
        "Kami adalah KitaFit, sebuah aplikasi inovatif yang memanfaatkan kecerdasan buatan untuk menganalisis dan memberikan wawasan kesehatan jantung Anda.",
        "Fokus utama KitaFit adalah pada pencegahan dan deteksi dini risiko penyakit jantung melalui pendekatan teknologi AI yang personal.",
        "KitaFit merupakan aplikasi mobile yang membantu Anda memantau kesehatan jantung dan memahami faktor-faktor risiko yang mungkin Anda miliki.",
        "Secara sederhana, KitaFit adalah asisten kesehatan digital Anda yang berfokus pada kesehatan jantung."
    ],
    "daftar": [
        "Anda bisa mendaftar dengan mudah melalui email atau nomor telepon. Cukup ikuti panduan pendaftaran di aplikasi.",
        "Proses pendaftaran KitaFit sangat sederhana. Anda hanya perlu menyiapkan email atau nomor telepon aktif untuk verifikasi.",
        "Untuk mulai menggunakan KitaFit, silakan buat akun baru melalui menu 'Daftar' di halaman awal aplikasi.",
        "Pendaftaran hanya butuh beberapa langkah mudah. Unduh aplikasi, pilih 'Daftar', dan ikuti instruksi di layar.",
        "Ya, pendaftaran di KitaFit gratis. Anda bisa langsung mendaftar untuk menikmati fitur-fitur dasar kami."
    ],
    "fitur": [
        "Fitur utama kami meliputi deteksi risiko penyakit jantung, konsultasi online dengan ahli (HeartConsult), dan rekomendasi gaya hidup personal.",
        "Di KitaFit, Anda bisa menikmati fitur seperti analisis risiko berbasis AI, sesi konsultasi virtual, serta beragam artikel kesehatan di LifeNews.",
        "Kami menyediakan fitur unggulan seperti deteksi dini risiko, sesi konsultasi online, dan panduan gaya hidup sehat yang disesuaikan untuk Anda.",
        "Anda dapat memanfaatkan fitur analisis risiko jantung, akses ke konsultan kesehatan, dan kumpulan artikel LifeNews untuk menjaga kesehatan Anda.",
        "Beberapa fitur andalan kami adalah HeartTrack untuk deteksi risiko, HeartConsult untuk konsultasi, dan LifeNews untuk informasi kesehatan terbaru."
    ],
    "biaya": [
        "KitaFit menyediakan fitur dasar secara gratis. Untuk fitur lebih lanjut seperti konsultasi tanpa batas, kami menawarkan paket premium berbayar.",
        "Anda dapat menggunakan KitaFit secara gratis untuk fitur-fitur esensial. Namun, beberapa fitur canggih memerlukan langganan premium.",
        "Kami menawarkan model 'freemium'. Anda bisa menikmati layanan dasar secara gratis dan bisa upgrade ke premium kapan saja untuk mendapatkan akses penuh.",
        "Untuk detail lengkap mengenai biaya langganan paket premium, Anda bisa mengeceknya langsung di menu 'Langganan' di dalam aplikasi.",
        "Tidak ada biaya untuk fitur dasar KitaFit. Biaya hanya dikenakan jika Anda memilih untuk berlangganan paket premium kami."
    ],
    "riwayat": [
        "Anda bisa melihat riwayat kesehatan dan semua hasil deteksi Anda di bagian 'Riwayat Kesehatan' di dalam aplikasi.",
        "Semua data kesehatan dan hasil deteksi Anda tersimpan aman dan dapat diakses kapan saja melalui menu 'Riwayat'.",
        "Untuk melihat histori kesehatan Anda, silakan kunjungi tab 'Riwayat' yang ada di dashboard utama aplikasi KitaFit.",
        "Riwayat deteksi dan informasi kesehatan Anda dapat ditemukan dengan mudah. Cukup buka aplikasi dan masuk ke menu riwayat.",
        "Ya, Anda bisa mengunduh laporan riwayat kesehatan Anda dalam format PDF melalui menu 'Riwayat'."
    ],
    "hasil": [
        "Hasil deteksi risiko disajikan dalam bentuk skor yang mudah dipahami, disertai penjelasan detail tentang kondisi Anda.",
        "Setelah proses deteksi selesai, Anda akan mendapatkan skor risiko, interpretasi, serta rekomendasi langkah selanjutnya.",
        "Laporan hasil deteksi mencakup skor probabilitas risiko dan analisis mendalam mengenai faktor-faktor yang memengaruhinya.",
        "Hasil deteksi akan muncul di layar Anda setelah analisis selesai, dan juga akan otomatis tersimpan di menu 'Riwayat'.",
        "Kami menyajikan hasil deteksi dengan visualisasi data yang jelas agar Anda mudah memahami kondisi kesehatan Anda."
    ],
    "reset_password": [
        "Untuk mengatur ulang password, silakan klik tombol 'Lupa Password' di halaman login, lalu ikuti instruksi yang dikirimkan ke email Anda.",
        "Jika Anda lupa kata sandi, gunakan fitur 'Lupa Password'. Kami akan mengirimkan tautan untuk reset ke email atau nomor telepon terdaftar Anda.",
        "Proses reset password dapat dilakukan melalui tautan yang akan kami kirimkan. Pastikan Anda memiliki akses ke email atau nomor telepon yang Anda gunakan saat mendaftar.",
        "Jangan khawatir jika lupa password. Cukup akses opsi 'Lupa Kata Sandi' di layar masuk untuk memulai proses pemulihan.",
        "Bantuan reset password tersedia di halaman login. Klik 'Lupa Password' dan kami akan memandu Anda melalui langkah-langkahnya."
    ],
    "konsultasi": [
        "Anda dapat menggunakan fitur HeartConsult kami untuk melakukan konsultasi online dengan para ahli kesehatan.",
        "Fitur konsultasi di KitaFit memungkinkan Anda terhubung langsung dengan dokter atau ahli gizi melalui chat atau video call.",
        "Manfaatkan layanan HeartConsult untuk mendapatkan saran medis atau gizi secara virtual dari para profesional terpercaya.",
        "Untuk memulai konsultasi, silakan masuk ke menu 'HeartConsult', pilih ahli yang Anda inginkan, dan jadwalkan sesi Anda.",
        "Tentu, Anda bisa memilih dokter spesialis jantung yang tersedia di platform kami untuk sesi konsultasi."
    ],
    "privasi": [
        "Keamanan data Anda adalah prioritas utama kami. Semua informasi pribadi di KitaFit dilindungi dengan enkripsi end-to-end.",
        "Kami berkomitmen penuh untuk menjaga kerahasiaan data pengguna. Informasi Anda tidak akan pernah dibagikan ke pihak ketiga tanpa izin eksplisit dari Anda.",
        "Data Anda aman dan disimpan secara terenkripsi. Kami menerapkan standar keamanan data tertinggi untuk melindungi privasi Anda.",
        "Kebijakan privasi kami menjelaskan secara transparan bagaimana data Anda dikumpulkan, digunakan, dan dilindungi secara ketat.",
        "Anda memiliki kontrol penuh atas data Anda. Anda bisa mengelola izin akses data melalui menu pengaturan privasi di aplikasi."
    ],
    "artikel": [
        "Anda bisa menemukan berbagai artikel dan berita terbaru seputar gaya hidup sehat di bagian 'LifeNews' dalam aplikasi.",
        "Silakan cek fitur 'LifeNews' untuk mendapatkan tips kesehatan, informasi nutrisi, dan panduan gaya hidup dari para ahli.",
        "Baca artikel-artikel informatif di 'LifeNews' KitaFit untuk meningkatkan pemahaman Anda tentang kesehatan jantung dan kebugaran.",
        "Kami menyediakan kumpulan artikel menarik dan edukatif di fitur 'LifeNews' yang diperbarui secara berkala oleh tim kami.",
        "Ya, semua artikel di LifeNews ditulis dan diverifikasi oleh para profesional di bidang kesehatan untuk memastikan akurasinya."
    ],
    "unknown": [
        "Maaf, saya tidak mengerti pertanyaan Anda. Bisakah Anda mengulanginya dengan cara yang berbeda atau coba tanyakan hal lain?",
        "Hmm, sepertinya pertanyaan itu di luar topik yang saya ketahui. Ada lagi yang bisa saya bantu seputar fitur, biaya, atau privasi di KitaFit?",
        "Saya belum dilatih untuk menjawab pertanyaan tersebut. Mungkin Anda bisa mencoba bertanya dengan kalimat yang lebih sederhana?",
        "Saya adalah bot yang fokus pada FAQ KitaFit. Untuk pertanyaan di luar itu, saya mungkin belum bisa menjawabnya."
    ]
}

# Fungsi untuk memprediksi intent dan memberikan respons
def get_chatbot_response(user_input, model, tfidf_vectorizer, label_encoder, intent_responses_dict, confidence_threshold=0.4):
    processed_input = preprocess_text(user_input)
    if not processed_input.strip():
        return random.choice(intent_responses_dict["unknown"]), "unknown", 0.0

    user_input_vector = tfidf_vectorizer.transform([processed_input]).toarray()
    predictions = model.predict(user_input_vector, verbose=0)

    predicted_intent_index = np.argmax(predictions)
    confidence = np.max(predictions)

    if confidence < confidence_threshold:
        predicted_intent_label = "unknown"
    else:
        predicted_intent_label = label_encoder.inverse_transform([predicted_intent_index])[0]
    
    possible_responses = intent_responses_dict.get(predicted_intent_label, intent_responses_dict["unknown"])
    response = random.choice(possible_responses)

    return response, predicted_intent_label, confidence

# --- Loop Interaktif Chatbot ---
print("\nKitaFit Bot: Halo! Saya KitaFit Bot, siap membantu Anda. (Ketik 'keluar' untuk mengakhiri)")
while True:
    user_input = input("Anda: ")
    
    if user_input.lower() == 'keluar':
        print("KitaFit Bot: Sampai jumpa! Senang bisa membantu Anda.")
        break
        
    if not user_input.strip():
        print("KitaFit Bot: Sepertinya Anda belum mengetik apa-apa. Ada yang bisa saya bantu?")
        continue

    response, intent, conf = get_chatbot_response(user_input, model, tfidf, le, intent_responses)
    
    print(f"Pertanyaan Anda: {user_input}")
    print(f"KitaFit Bot: {response} (Intent: {intent}, Confidence: {conf:.2f})")
    print("-" * 30)


KitaFit Bot: Halo! Saya KitaFit Bot, siap membantu Anda. (Ketik 'keluar' untuk mengakhiri)
Pertanyaan Anda: KitaFit itu aplikasi apa?
KitaFit Bot: Kami adalah KitaFit, sebuah aplikasi inovatif yang memanfaatkan kecerdasan buatan untuk menganalisis dan memberikan wawasan kesehatan jantung Anda. (Intent: kitafit, Confidence: 1.00)
------------------------------
Pertanyaan Anda: Bagaimana cara mendaftar di KitaFit?
KitaFit Bot: Untuk mulai menggunakan KitaFit, silakan buat akun baru melalui menu 'Daftar' di halaman awal aplikasi. (Intent: daftar, Confidence: 1.00)
------------------------------
Pertanyaan Anda: Apa saja fitur yang ada di KitaFit?
KitaFit Bot: Anda dapat memanfaatkan fitur analisis risiko jantung, akses ke konsultan kesehatan, dan kumpulan artikel LifeNews untuk menjaga kesehatan Anda. (Intent: fitur, Confidence: 1.00)
------------------------------
KitaFit Bot: Sepertinya Anda belum mengetik apa-apa. Ada yang bisa saya bantu?
Pertanyaan Anda: Berapa biaya menggunakan Kit